<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_2)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to define a thread**

In [ ]:
import threading

In [ ]:
def function(i):
  print("function called by thread %i\n" %i)
  return

In [ ]:
threads = []
for i in range(5):
  t = threading.Thread(target=function, args=(i,))
  threads.append(t)
  t.start()
  t.join()

function called by thread 0

function called by thread 1

function called by thread 2

function called by thread 3

function called by thread 4



**How to determine the current thread**

In [ ]:
import threading
import time

def first_function():
  print(threading.currentThread().getName()+ str(' is Starting \n'))
  time.sleep(2)
  print(threading.currentThread().getName() + str(' is Exiting \n'))
  return

def second_function():
  print(threading.currentThread().getName() + str(' is Starting \n'))
  time.sleep(2)
  print(threading.currentThread().getName() + str(' is Exiting \n'))
  return

def third_function():
  print(threading.currentThread().getName() + str('is Staring \n'))
  time.sleep(2)
  print(threading.currentThread().getName() + str('is Exiting \n'))
  return

t1 = threading.Thread(name='first_function', target=first_function)
t2 = threading.Thread(name='second_function', target=second_function)
t3 = threading.Thread(name='third_function', target=third_function)

t1.start()
t2.start()
t3.start()

# Т.к в коллабе при завершении основного потока закрываются остальные потоки не дожидаясь завершения,
# то добавим ожидание остальныех потоков с помощью метода .join(), чтобы получить ожидаемый вывод
t1.join() 
t2.join()
t3.join()

first_function is Starting 

second_function is Starting 

third_functionis Staring 

second_function is Exiting 

third_functionis Exiting 

first_function is Exiting 



**How to use a thread in a subclass**

In [ ]:
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
  def __init__(self, threadID, name, counter):
    threading.Thread.__init__(self)
    self.threadID = threadID
    self.name = name
    self.counter = counter

  def run(self):
    print("Starting " + self.name)
    print_time(self.name, self.counter, 5)
    print ("Exiting " + self.name)

In [ ]:
def print_time(threadName, delay, counter):
  while counter:
    if exitFlag: 
      thread.exit()
    time.sleep(delay)
    print("%s: %s" %(threadName, time.ctime(time.time())))
    counter -= 1

In [ ]:
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

thread1.start()
thread2.start()

thread1.join()
thread2.join()
print("Exiting Main Thread")

Starting Thread-1
Starting Thread-2
Thread-1: Mon Nov 15 18:55:24 2021
Thread-1: Mon Nov 15 18:55:25 2021
Thread-2: Mon Nov 15 18:55:25 2021
Thread-1: Mon Nov 15 18:55:26 2021
Thread-1: Mon Nov 15 18:55:27 2021
Thread-2: Mon Nov 15 18:55:27 2021
Thread-1: Mon Nov 15 18:55:28 2021
Exiting Thread-1
Thread-2: Mon Nov 15 18:55:29 2021
Thread-2: Mon Nov 15 18:55:31 2021
Thread-2: Mon Nov 15 18:55:33 2021
Exiting Thread-2
Exiting Main Thread


**Thread synchronization with Lock and
RLock**

In [14]:
import threading

shared_resource_with_lock = 0
shared_resource_with_no_lock = 0
# количество итераций увеличено, по сравнению с ориганиалом кода в учебнике, поскольку при малом количестве итераций
# периодически выдавался одинаковый результат при наличии и отсутствии блокировок примитива синхронизации.
COUNT = 300000
shared_resource_lock = threading.Lock()

In [9]:
def increment_with_lock():
    global shared_resource_with_lock
    for i in range(COUNT):
        shared_resource_lock.acquire()
        shared_resource_with_lock += 1
        shared_resource_lock.release()

In [10]:
def decrement_with_lock():
    global shared_resource_with_lock
    for i in range(COUNT):
        shared_resource_lock.acquire()
        shared_resource_with_lock -= 1
        shared_resource_lock.release()

In [11]:
def increment_without_lock():
    global shared_resource_with_no_lock
    for i in range(COUNT):
        shared_resource_with_no_lock += 1


In [12]:
def decrement_without_lock():
    global shared_resource_with_no_lock
    for i in range(COUNT):
        shared_resource_with_no_lock -= 1

In [18]:
t1 = threading.Thread(target=increment_with_lock)
t2 = threading.Thread(target=decrement_with_lock)
t3 = threading.Thread(target=increment_without_lock)
t4 = threading.Thread(target=decrement_without_lock)

t1.start()
t2.start()
t3.start()
t4.start()

t1.join()
t2.join()
t3.join()
t4.join()

print("the value of shared variable with lock management is", shared_resource_with_lock)
print("the value of shared variable with race condition is", shared_resource_with_no_lock)

the value of shared variable with lock management is 0
the value of shared variable with race condition is -127867


**Thread synchronization with RLock**

Главное отличие от Lock() заключается в том, что с помощью Lock() можно захватить ресурс только один раз. Его нельзя захватить снова, пока он не будет освобожден. (После того, как он был освобожден, он может быть повторно захвачен любым потоком).

С другой стороны, RLock может быть захвачен несколько раз одним и тем же потоком. Чтобы разблокировать Rlock(), его нужно освободить одинаковое количество раз.

Ещё одно отличие состоит в том, что ресурс захваченный Lock() может быть освобожден любым потоком, в то время как ресурс захваченный RLock() может быть освобожден только тем потоком, который ее захватил.

In [32]:
import threading
import time

In [37]:
class Box(object):
  lock = threading.RLock()
  def __init__(self):
    self.total_items = 0

  def execute (self,n):
    Box.lock.acquire()
    self.total_items += n
    Box.lock.release()
  
  def add(self):
    Box.lock.acquire()
    self.execute(1)
    Box.lock.release()

  def remove(self):
    Box.lock.acquire()
    self.execute(-1)
    Box.lock.release()

In [38]:
def adder(box, items):
  while items > 0:
    print ("adding 1 item in the box\n")
    box.add()
    time.sleep(5)
    items -= 1

In [39]:
def remover(box, items):
  while items > 0:
    print("removing 1 item in the box")
    box.remove()
    time.sleep(5)
    items -= 1

In [40]:
items = 5
print("putting %s items in the box " % items)
box = Box()
t1 = threading.Thread(target=adder, args=(box,items))
t2 = threading.Thread(target=remover,args=(box,items))
t1.start()
t2.start()

t1.join()
t2.join()
print("%s items still remain in the box " % box.total_items)

putting 5 items in the box 
adding 1 item in the box

removing 1 item in the box
adding 1 item in the box

removing 1 item in the box
adding 1 item in the box

removing 1 item in the box
adding 1 item in the box

removing 1 item in the box
adding 1 item in the box

removing 1 item in the box
0 items still remain in the box 


**Thread synchronization with semaphores**

In [16]:
import threading
import time 
import random

semaphore = threading.Semaphore(0)


In [17]:
def producer():
  global item
  time.sleep(10)
  item = random.randint(0,1000)
  print("Producer notify : produced item number %s" %item)
  semaphore.release() # происходит увеличение значение семафора на 1

In [18]:
def consumer():
  print("consumer is waiting.")
  semaphore.acquire() # поток блокируется до тех пор, пока счётчик не будет увеличен 
  print("Consumer notify : consumed item number %s" %item)

In [20]:
for i in range(0,5):
  t1 = threading.Thread(target=producer)
  t2 = threading.Thread(target=consumer)
  t1.start()
  t2.start()
  t1.join()
  t2.join()
print("program terminated")

consumer is waiting.
Producer notify : produced item number 919
Consumer notify : consumed item number 919
consumer is waiting.
Producer notify : produced item number 29
Consumer notify : consumed item number 29
consumer is waiting.
Producer notify : produced item number 594
Consumer notify : consumed item number 594
consumer is waiting.
Producer notify : produced item number 111
Consumer notify : consumed item number 111
consumer is waiting.
Producer notify : produced item number 135
Consumer notify : consumed item number 135
program terminated


**Thread synchronization with a condition**

In [1]:
from threading import Thread, Condition
import time

items = []
condition = Condition()

class consumer(Thread):
  def __init(self):
    Thread.__init__(self)

  def consume(self):
    global condition
    global items

    condition.acquire()
    if len(items) == 0:
      condition.wait()
      print("Consumer notify: no item to consume")
    items.pop()
    print("Consumer notify: consumed 1 item")
    print("Consumer notify: items to consume are " + str(len(items)))
    condition.notify()
    condition.release()

  def run(self):
    for i in range(0,20):
      time.sleep(10)
      self.consume()

class producer(Thread):
  def __init__(self):
    Thread.__init__(self)

  def produce(self):
    global condition
    global items

    condition.acquire()
    if len(items) == 10:
      condition.wait()
      print("Producer notify: items producted are " + str(len(items)))
      print("Producer notify: stop the production!!")
    items.append(1)
    print("Producer notify: total items producted " + str(len(items)))
    condition.notify()
    condition.release()

  def run(self):
    for i in range(0,20):
      time.sleep(5)
      self.produce() 

producer = producer()
consumer = consumer()

producer.start()
consumer.start()

producer.join()
consumer.join()




Producer notify: total items producted 1
Consumer notify: consumed 1 item
Consumer notify: items to consume are 0
Producer notify: total items producted 1
Producer notify: total items producted 2
Consumer notify: consumed 1 item
Consumer notify: items to consume are 1
Producer notify: total items producted 2
Producer notify: total items producted 3
Consumer notify: consumed 1 item
Consumer notify: items to consume are 2
Producer notify: total items producted 3
Producer notify: total items producted 4
Consumer notify: consumed 1 item
Consumer notify: items to consume are 3
Producer notify: total items producted 4
Producer notify: total items producted 5
Consumer notify: consumed 1 item
Consumer notify: items to consume are 4
Producer notify: total items producted 5
Producer notify: total items producted 6
Consumer notify: consumed 1 item
Consumer notify: items to consume are 5
Producer notify: total items producted 6
Producer notify: total items producted 7
Consumer notify: consumed 1 i

**Thread synchronization with an event**